In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import skimage as sk
from skimage import measure
import os
import tifffile
from tqdm import tqdm

In [2]:
dots_data = pd.read_csv("/mnt/data/Imaging/202105-Deconwolf/data_210726/dots_data.clean.tsv.gz", sep="\t")

In [3]:
dots_data2 = dots_data.loc["60x" == dots_data["magnification"], :]

In [4]:
matched_dots = pd.read_csv("../../data/60x_matched_dots.tsv", sep="\t")

In [5]:
ref_raw = dots_data2.loc["raw" == dots_data2["image_type"], :].reset_index(drop=True)
ref__dw = dots_data2.loc["dw" == dots_data2["image_type"], :].reset_index(drop=True)
raw_image_folder_path = "/mnt/data/Imaging/202105-Deconwolf/data_210726/60x_raw"
dw__image_folder_path = "/mnt/data/Imaging/202105-Deconwolf/data_210726/60x_dw"
mask_folder_path = "../../data/60x_mask/dilated_labels_watershed"
field_list = np.unique(dots_data2["sid"].values).tolist()
raw_thr = 0.15
dw__thr = 5.3

In [6]:
for current_field_id in field_list:
    print(f"Field #{current_field_id}")

    raw_max_z_proj = tifffile.imread(os.path.join(raw_image_folder_path, f"a647_{current_field_id:03d}.tif")).max(0)
    dw__max_z_proj = tifffile.imread(os.path.join(dw__image_folder_path, f"a647_{current_field_id:03d}.tif")).max(0)
    
    labels = tifffile.imread(os.path.join(mask_folder_path, f"a647_{current_field_id:03d}.dilated_labels.tiff")).reshape(raw_max_z_proj.shape)

    field_raw_dots = ref_raw.loc[ref_raw["sid"] == current_field_id, :].sort_values("Value2", ascending=False)
    field_dw__dots = ref__dw.loc[ref__dw["sid"] == current_field_id, :].sort_values("Value2", ascending=False)
    
    matched_raw_dots = matched_dots.loc[matched_dots["series"] == current_field_id, "raw_id"].values
    matched_dw__dots = matched_dots.loc[matched_dots["series"] == current_field_id, "dw__id"].values
    max_match_dist = matched_dots.loc[matched_dots["series"] == current_field_id, "eudist"].max()
    
    selected_raw_dots = field_raw_dots.loc[field_raw_dots["Value2"] >= raw_thr, :].reset_index(drop=True).iloc[matched_raw_dots, :]
    selected_dw__dots = field_dw__dots.loc[field_dw__dots["Value2"] >= dw__thr, :].reset_index(drop=True).iloc[matched_dw__dots, :]

    fig3, ax = plt.subplots(figsize=(30, 10), ncols=3, constrained_layout=True)
    fig3.suptitle(f"Field #{current_field_id} (n.dots={matched_raw_dots.shape[0]}; max.dist={max_match_dist:.03f})")
    print(" > Plotting raw")
    ax[0].set_title("60x_raw")
    ax[0].imshow(
        raw_max_z_proj, cmap=plt.get_cmap("gray"), interpolation="none",
        vmin=raw_max_z_proj.min(), vmax=raw_max_z_proj.max(),
        resample=False, filternorm=False)
    ax[0].scatter(
        x=selected_raw_dots["y"].values,
        y=selected_raw_dots["x"].values,
        s=30, facecolors='none', edgecolors='r', linewidth=.5)
    print(" > Plotting dw")
    ax[1].set_title("60x_dw")
    ax[1].imshow(
        dw__max_z_proj, cmap=plt.get_cmap("gray"), interpolation="none",
        vmin=dw__max_z_proj.min()*1.5, vmax=dw__max_z_proj.max()*.5,
        resample=False, filternorm=False)
    ax[1].scatter(
        x=selected_dw__dots["y"].values,
        y=selected_dw__dots["x"].values,
        s=30, facecolors='none', edgecolors='r', linewidth=.5)
    print(" > Plotting contours")
    for lid in tqdm(range(1, labels.max()), desc="nucleus"):
        contours = measure.find_contours(labels == lid, 0.8)
        for contour in contours:
            ax[0].scatter(x=contour[:,1], y=contour[:,0], c="yellow", s=.005)
            ax[1].scatter(x=contour[:,1], y=contour[:,0], c="yellow", s=.005)
    print(" > Plotting overlapped points between raw and dw")
    ax[2].set_title("Red: 60x_raw. Blue: 60x_dw.")
    ax[2].plot(
        selected_raw_dots["y"].values,
        selected_raw_dots["x"].values,
        'r.', marker=".", markersize=2)
    ax[2].plot(
        selected_dw__dots["y"].values,
        selected_dw__dots["x"].values,
        'b.', marker=".", markersize=.8)
    plt.close(fig3)
    #print(" > Exporting")
    fig3.savefig(f"../../data/60x_reference_overlay/overlay_{current_field_id:03d}.png", bbox_inches='tight')
    print(" ! DONE")

Field #1


nucleus:   1%|          | 2/180 [00:00<00:11, 14.86it/s]

 > Plotting raw
 > Plotting dw
 > Plotting contours


nucleus: 100%|██████████| 180/180 [00:11<00:00, 15.42it/s]


 > Plotting overlapped points between raw and dw
 ! DONE
Field #2


...

 > Plotting overlapped points between raw and dw
 ! DONE
Field #7


nucleus:   1%|          | 2/221 [00:00<00:13, 15.71it/s]

 > Plotting raw
 > Plotting dw
 > Plotting contours


nucleus: 100%|██████████| 221/221 [00:13<00:00, 16.12it/s]


 > Plotting overlapped points between raw and dw
 ! DONE
